In [2]:
pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [3]:
import random
import csv

def generate_matrix(length, min_value, max_value, seed):
    random.seed(seed)
    matrix = []
    for i in range(length):
        row = []
        for j in range(length):
            if i == j:
                row.append(0)
            else:
                row.append(random.randint(min_value, max_value))
        matrix.append(row)
    return matrix

def export_matrix_to_csv(matrix, filename):
    # Create a copy of the matrix with diagonal values set to 9999
    matrix_with_9999 = []
    for i in range(len(matrix)):
        row = []
        for j in range(len(matrix[i])):
            if i == j:
                row.append(9999)
            else:
                row.append(matrix[i][j])
        matrix_with_9999.append(row)

    # Write the copied matrix to a CSV file
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(matrix_with_9999)
    print(f"Matrix has been exported to {filename}")

# User input
length = 11
min_value = 100
max_value = 999
j= 0
sd=42
matrices=[]
while j < 4:
    filename = input("Enter the filename (with .csv extension): ")

    # Generate the matrix
    matrix = generate_matrix(length, min_value, max_value, seed=sd)
    print("Random Square Matrix with Zero Diagonal:")
    for row in matrix:
        print(row)
    matrices.append(matrix)

    # Export the matrix to a CSV file with diagonal values set to 9999
    export_matrix_to_csv(matrix, filename)
    sd+=1
    j+=1

Enter the filename (with .csv extension): Matrix11-2.csv
Random Square Matrix with Zero Diagonal:
[0, 754, 214, 125, 859, 381, 350, 328, 242, 854, 204]
[792, 0, 858, 658, 189, 704, 532, 132, 130, 195, 323]
[338, 617, 0, 716, 127, 674, 303, 833, 765, 818, 658]
[529, 325, 559, 0, 703, 384, 928, 990, 106, 877, 925]
[263, 814, 532, 448, 0, 384, 259, 320, 881, 444, 204]
[194, 489, 199, 467, 967, 0, 452, 718, 370, 926, 144]
[847, 570, 649, 227, 487, 180, 0, 665, 400, 949, 743]
[733, 982, 470, 691, 296, 821, 171, 0, 146, 777, 333]
[891, 396, 181, 975, 338, 987, 203, 489, 0, 384, 564]
[750, 954, 473, 266, 479, 463, 314, 786, 373, 0, 818]
[799, 763, 173, 723, 750, 275, 646, 846, 350, 267, 0]
Matrix has been exported to Matrix11-2.csv
Enter the filename (with .csv extension): Matrix11-3.csv
Random Square Matrix with Zero Diagonal:
[0, 139, 392, 812, 880, 247, 573, 478, 787, 815, 911]
[198, 0, 564, 713, 610, 723, 119, 626, 542, 690, 481]
[739, 972, 0, 660, 873, 544, 503, 280, 159, 971, 209]
[939,

#Primer caso 11 nodos

In [4]:
%%time

"""Capacited Vehicles Routing Problem (CVRP)."""
import psutil
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0,808,654,275,134,357,545,236,123,484,645],
      [808,0,156,724,752,860,960,870,849,1036,1207],
      [654,156,0,571,596,710,854,731,701,889,1063],
      [275,724,571,0,141,151,796,503,396,331,511],
      [134,752,596,141,0,242,663,364,255,396,570],
      [357,860,710,151,242,0,900,593,478,181,360],
      [545,960,854,796,663,900,0,313,429,1023,1171],
      [236,870,731,503,364,593,313,0,116,710,859],
      [123,849,701,396,255,478,429,116,0,595,746],
      [484,1036,889,331,396,181,1023,710,595,0,179],
      [645,1207,1063,511,570,360,1171,859,746,179,0],
        # fmt: on
    ]
    data["demands"] = [0,40,56,64,11,92,38,53,64,5,75]
    data["vehicle_capacities"] = [100,100,100,100,100,100]
    data["num_vehicles"] = 6
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"SOLUTION FOUND...\n\n\n")
    total_distance = 0
    total_load = 0
    capacity= data["vehicle_capacities"][0]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Remaining({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Remaining({route_load})\n"
        plan_output += f"Time of the route: {route_distance/100} minutes\n"
        plan_output += f"Load of the route: {route_load/capacity}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Objective: {solution.ObjectiveValue()/100+total_load/capacity*30} minutes in total.")
    print(f"Total time of all routes: {total_distance/100} minutes")
    print(f"Total load of all routes: {total_load/capacity}")
    print(f"Total time spent loading and unloading: {total_load/capacity*30} minutes")

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    #Se utiliza en primera instancia una heurística , que comienza por un nodo inicial que por consiguiente busca otro nodo , haciendo una ruta de la manera más económica o más barata, y luego
    #extiende a otro nodo , extendiendo la ruta con la misma lógica
    #Asi mismo se utiliza una metaheurística para mejorar la solución inicial , usamos guided local search es un proceso iterativo que busca mejores soluciones  a  la inicial y se continua el proceso hasta no mejorar la situación actual
    #Es parecio a que local search  a diferencia es que en guided local search para poder salir de óptimos locales , se agregan penalizaciones, lo que lo hace más efectivo.
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

RAM usage: 146.46484375 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  2 Remaining(56) ->  1 Remaining(96) ->  0 Remaining(96)
Time of the route: 16.18 minutes
Load of the route: 0.96

Route for vehicle 1:
 0 Remaining(0) ->  9 Remaining(5) ->  10 Remaining(80) ->  0 Remaining(80)
Time of the route: 13.08 minutes
Load of the route: 0.8

Route for vehicle 2:
 0 Remaining(0) ->  5 Remaining(92) ->  0 Remaining(92)
Time of the route: 7.14 minutes
Load of the route: 0.92

Route for vehicle 3:
 0 Remaining(0) ->  4 Remaining(11) ->  3 Remaining(75) ->  0 Remaining(75)
Time of the route: 5.5 minutes
Load of the route: 0.75

Route for vehicle 4:
 0 Remaining(0) ->  7 Remaining(53) ->  6 Remaining(91) ->  0 Remaining(91)
Time of the route: 10.94 minutes
Load of the route: 0.91

Route for vehicle 5:
 0 Remaining(0) ->  8 Remaining(64) ->  0 Remaining(64)
Time of the route: 2.46 minutes
Load of the route: 0.64

Objective: 204.7 minutes in total.
Total time of all routes: 55.3 m

#Segundo caso 11 nodos

In [5]:
%%time
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"]= [
        [0, 754, 214, 125, 859, 381, 350, 328, 242, 854, 204],
        [792, 0, 858, 658, 189, 704, 532, 132, 130, 195, 323],
        [338, 617, 0, 716, 127, 674, 303, 833, 765, 818, 658],
        [529, 325, 559, 0, 703, 384, 928, 990, 106, 877, 925],
        [263, 814, 532, 448, 0, 384, 259, 320, 881, 444, 204],
        [194, 489, 199, 467, 967, 0, 452, 718, 370, 926, 144],
        [847, 570, 649, 227, 487, 180, 0, 665, 400, 949, 743],
        [733, 982, 470, 691, 296, 821, 171, 0, 146, 777, 333],
        [891, 396, 181, 975, 338, 987, 203, 489, 0, 384, 564],
        [750, 954, 473, 266, 479, 463, 314, 786, 373, 0, 818],
        [799, 763, 173, 723, 750, 275, 646, 846, 350, 267, 0],
    ]
    data["demands"] = [0,52,52,11,54,23,31,10,30,62,66]
    data["vehicle_capacities"] = [100,100,100,100,100]
    data["num_vehicles"] = 5
    data["depot"] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"SOLUTION FOUND...\n\n\n")
    total_distance = 0
    total_load = 0
    capacity= data["vehicle_capacities"][0]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Remaining({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Remaining({route_load})\n"
        plan_output += f"Time of the route: {route_distance} minutes\n"
        plan_output += f"Load of the route: {route_load/capacity}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Objective: {solution.ObjectiveValue()+total_load/capacity*30} minutes in total.")
    print(f"Total time of all routes: {total_distance} minutes")
    print(f"Total load of all routes: {total_load/capacity}")
    print(f"Total time spent loading and unloading: {total_load/capacity*30} minutes")
if __name__ == "__main__":
    main()

RAM usage: 149.41796875 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  8 Remaining(30) ->  9 Remaining(92) ->  0 Remaining(92)
Time of the route: 1376 minutes
Load of the route: 0.92

Route for vehicle 1:
 0 Remaining(0) ->  3 Remaining(11) ->  1 Remaining(63) ->  7 Remaining(73) ->  0 Remaining(73)
Time of the route: 1315 minutes
Load of the route: 0.73

Route for vehicle 2:
 0 Remaining(0) ->  6 Remaining(31) ->  4 Remaining(85) ->  0 Remaining(85)
Time of the route: 1100 minutes
Load of the route: 0.85

Route for vehicle 3:
 0 Remaining(0) ->  10 Remaining(66) ->  5 Remaining(89) ->  0 Remaining(89)
Time of the route: 673 minutes
Load of the route: 0.89

Route for vehicle 4:
 0 Remaining(0) ->  2 Remaining(52) ->  0 Remaining(52)
Time of the route: 552 minutes
Load of the route: 0.52

Objective: 5133.3 minutes in total.
Total time of all routes: 5016 minutes
Total load of all routes: 3.91
Total time spent loading and unloading: 117.30000000000001 minutes
CPU times:

#Tercer caso 11 nodos

In [6]:
matrices[1]

[[0, 139, 392, 812, 880, 247, 573, 478, 787, 815, 911],
 [198, 0, 564, 713, 610, 723, 119, 626, 542, 690, 481],
 [739, 972, 0, 660, 873, 544, 503, 280, 159, 971, 209],
 [939, 227, 484, 0, 257, 198, 264, 624, 954, 607, 231],
 [890, 496, 711, 618, 0, 378, 688, 286, 485, 165, 150],
 [693, 162, 291, 162, 959, 0, 269, 403, 381, 178, 634],
 [738, 205, 976, 933, 698, 769, 0, 765, 795, 162, 196],
 [965, 737, 568, 905, 762, 840, 439, 0, 438, 641, 780],
 [817, 549, 270, 229, 968, 182, 899, 427, 0, 151, 997],
 [705, 374, 476, 845, 290, 788, 718, 172, 665, 0, 778],
 [216, 404, 606, 897, 846, 470, 832, 544, 142, 584, 0]]

In [12]:
%%time
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"]= [
        [0, 139, 392, 812, 880, 247, 573, 478, 787, 815, 911],
        [198, 0, 564, 713, 610, 723, 119, 626, 542, 690, 481],
        [739, 972, 0, 660, 873, 544, 503, 280, 159, 971, 209],
        [939, 227, 484, 0, 257, 198, 264, 624, 954, 607, 231],
        [890, 496, 711, 618, 0, 378, 688, 286, 485, 165, 150],
        [693, 162, 291, 162, 959, 0, 269, 403, 381, 178, 634],
        [738, 205, 976, 933, 698, 769, 0, 765, 795, 162, 196],
        [965, 737, 568, 905, 762, 840, 439, 0, 438, 641, 780],
        [817, 549, 270, 229, 968, 182, 899, 427, 0, 151, 997],
        [705, 374, 476, 845, 290, 788, 718, 172, 665, 0, 778],
        [216, 404, 606, 897, 846, 470, 832, 544, 142, 584, 0],
        ]

    data["demands"] = [0,79,65,62,35,14,77,74,75,93,25]
    data["vehicle_capacities"] = [100,100,100,100,100,100,100]
    data["num_vehicles"] = 7
    data["depot"] = 0
    return data

if __name__ == "__main__":
    main()

RAM usage: 149.41796875 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  9 Remaining(93) ->  0 Remaining(93)
Time of the route: 1520 minutes
Load of the route: 0.93

Route for vehicle 1:
 0 Remaining(0) ->  3 Remaining(62) ->  4 Remaining(97) ->  0 Remaining(97)
Time of the route: 1959 minutes
Load of the route: 0.97

Route for vehicle 2:
 0 Remaining(0) ->  5 Remaining(14) ->  8 Remaining(89) ->  0 Remaining(89)
Time of the route: 1445 minutes
Load of the route: 0.89

Route for vehicle 3:
 0 Remaining(0) ->  6 Remaining(77) ->  0 Remaining(77)
Time of the route: 1311 minutes
Load of the route: 0.77

Route for vehicle 4:
 0 Remaining(0) ->  7 Remaining(74) ->  0 Remaining(74)
Time of the route: 1443 minutes
Load of the route: 0.74

Route for vehicle 5:
 0 Remaining(0) ->  2 Remaining(65) ->  10 Remaining(90) ->  0 Remaining(90)
Time of the route: 817 minutes
Load of the route: 0.9

Route for vehicle 6:
 0 Remaining(0) ->  1 Remaining(79) ->  0 Remaining(79)
Time of the 

#Cuarto caso 11 nodos

In [8]:
matrices[2]

[[0, 518, 632, 654, 818, 982, 219, 280, 488, 330, 396],
 [129, 0, 330, 219, 682, 109, 202, 260, 893, 626, 805],
 [410, 810, 0, 749, 487, 705, 514, 884, 807, 786, 433],
 [466, 175, 446, 0, 961, 389, 732, 262, 214, 623, 171],
 [702, 196, 376, 422, 0, 210, 740, 941, 396, 135, 452],
 [495, 198, 588, 587, 295, 0, 293, 233, 364, 865, 353],
 [895, 150, 144, 789, 936, 796, 0, 546, 826, 138, 856],
 [830, 811, 854, 964, 769, 930, 436, 0, 478, 950, 582],
 [745, 564, 704, 921, 653, 254, 338, 595, 0, 566, 913],
 [854, 800, 454, 731, 939, 959, 855, 144, 384, 0, 261],
 [109, 328, 114, 693, 871, 718, 762, 669, 656, 388, 0]]

In [13]:
%%time
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"]= [
        [0, 518, 632, 654, 818, 982, 219, 280, 488, 330, 396],
        [129, 0, 330, 219, 682, 109, 202, 260, 893, 626, 805],
        [410, 810, 0, 749, 487, 705, 514, 884, 807, 786, 433],
        [466, 175, 446, 0, 961, 389, 732, 262, 214, 623, 171],
        [702, 196, 376, 422, 0, 210, 740, 941, 396, 135, 452],
        [495, 198, 588, 587, 295, 0, 293, 233, 364, 865, 353],
        [895, 150, 144, 789, 936, 796, 0, 546, 826, 138, 856],
        [830, 811, 854, 964, 769, 930, 436, 0, 478, 950, 582],
        [745, 564, 704, 921, 653, 254, 338, 595, 0, 566, 913],
        [854, 800, 454, 731, 939, 959, 855, 144, 384, 0, 261],
        [109, 328, 114, 693, 871, 718, 762, 669, 656, 388, 0],
        ]

    data["demands"] = [0,55,54,71,66,19,37,25,35,86,32]
    data["vehicle_capacities"] = [100,100,100,100,100,100]
    data["num_vehicles"] = 6
    data["depot"] = 0
    return data

if __name__ == "__main__":
    main()

RAM usage: 149.41796875 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  4 Remaining(66) ->  10 Remaining(98) ->  0 Remaining(98)
Time of the route: 1379 minutes
Load of the route: 0.98

Route for vehicle 1:
 0 Remaining(0) ->  3 Remaining(71) ->  0 Remaining(71)
Time of the route: 1120 minutes
Load of the route: 0.71

Route for vehicle 2:
 0 Remaining(0) ->  1 Remaining(55) ->  0 Remaining(55)
Time of the route: 647 minutes
Load of the route: 0.55

Route for vehicle 3:
 0 Remaining(0) ->  9 Remaining(86) ->  0 Remaining(86)
Time of the route: 1184 minutes
Load of the route: 0.86

Route for vehicle 4:
 0 Remaining(0) ->  7 Remaining(25) ->  8 Remaining(60) ->  5 Remaining(79) ->  0 Remaining(79)
Time of the route: 1507 minutes
Load of the route: 0.79

Route for vehicle 5:
 0 Remaining(0) ->  6 Remaining(37) ->  2 Remaining(91) ->  0 Remaining(91)
Time of the route: 773 minutes
Load of the route: 0.91

Objective: 6754.0 minutes in total.
Total time of all routes: 6610 mi

#Quinto caso 11 nodos

In [10]:
matrices[3]

[[0, 378, 527, 599, 363, 183, 410, 446, 122, 174, 595],
 [111, 0, 217, 390, 927, 220, 418, 975, 861, 764, 136],
 [739, 168, 0, 238, 378, 914, 854, 257, 316, 968, 878],
 [834, 538, 176, 0, 727, 517, 246, 817, 158, 132, 276],
 [428, 357, 465, 893, 0, 842, 427, 196, 521, 191, 121],
 [869, 436, 424, 902, 496, 0, 815, 997, 863, 643, 776],
 [649, 988, 747, 849, 735, 929, 0, 819, 758, 360, 707],
 [779, 161, 604, 482, 314, 459, 259, 0, 741, 907, 117],
 [355, 611, 828, 738, 534, 400, 314, 524, 0, 494, 955],
 [489, 246, 698, 566, 667, 188, 673, 493, 163, 0, 627],
 [727, 129, 739, 979, 656, 774, 745, 633, 890, 395, 0]]

In [14]:
%%time
print('RAM usage:', psutil.Process().memory_info().rss / 1024 ** 2, 'MB')

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"]= [
        [0, 378, 527, 599, 363, 183, 410, 446, 122, 174, 595],
        [111, 0, 217, 390, 927, 220, 418, 975, 861, 764, 136],
        [739, 168, 0, 238, 378, 914, 854, 257, 316, 968, 878],
        [834, 538, 176, 0, 727, 517, 246, 817, 158, 132, 276],
        [428, 357, 465, 893, 0, 842, 427, 196, 521, 191, 121],
        [869, 436, 424, 902, 496, 0, 815, 997, 863, 643, 776],
        [649, 988, 747, 849, 735, 929, 0, 819, 758, 360, 707],
        [779, 161, 604, 482, 314, 459, 259, 0, 741, 907, 117],
        [355, 611, 828, 738, 534, 400, 314, 524, 0, 494, 955],
        [489, 246, 698, 566, 667, 188, 673, 493, 163, 0, 627],
        [727, 129, 739, 979, 656, 774, 745, 633, 890, 395, 0],
        ]

    data["demands"] = [0,85,12,46,13,6,83,6,70,69,25]
    data["vehicle_capacities"] = [100,100,100,100,100]
    data["num_vehicles"] = 5
    data["depot"] = 0
    return data

if __name__ == "__main__":
    main()

RAM usage: 149.671875 MB
SOLUTION FOUND...



Route for vehicle 0:
 0 Remaining(0) ->  5 Remaining(6) ->  2 Remaining(18) ->  3 Remaining(64) ->  10 Remaining(89) ->  0 Remaining(89)
Time of the route: 1848 minutes
Load of the route: 0.89

Route for vehicle 1:
 0 Remaining(0) ->  4 Remaining(13) ->  6 Remaining(96) ->  0 Remaining(96)
Time of the route: 1439 minutes
Load of the route: 0.96

Route for vehicle 2:
 0 Remaining(0) ->  7 Remaining(6) ->  1 Remaining(91) ->  0 Remaining(91)
Time of the route: 718 minutes
Load of the route: 0.91

Route for vehicle 3:
 0 Remaining(0) ->  9 Remaining(69) ->  0 Remaining(69)
Time of the route: 663 minutes
Load of the route: 0.69

Route for vehicle 4:
 0 Remaining(0) ->  8 Remaining(70) ->  0 Remaining(70)
Time of the route: 477 minutes
Load of the route: 0.7

Objective: 5269.5 minutes in total.
Total time of all routes: 5145 minutes
Total load of all routes: 4.15
Total time spent loading and unloading: 124.50000000000001 minutes
CPU times: user 